# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [8]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [9]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port alfred,-33.5906,26.8910,52.18,82,89,5.99,ZA,1690155180
1,1,bredasdorp,-34.5322,20.0403,49.69,72,59,5.97,ZA,1690155179
2,2,suva,-18.1416,178.4415,78.28,69,75,3.44,FJ,1690155228
3,3,nagqu,31.5000,92.0000,39.96,94,100,3.78,CN,1690155203
4,4,thompson,55.7435,-97.8558,71.76,40,20,12.66,CA,1690155179


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [10]:
%%capture --no-display

# Configure the map plot
city_map = city_data_df.hvplot.points("Lng", 
                                      "Lat", 
                                      geo = True,
                                      color = "City",
                                      alpha = 0.6,
                                      size = "Humidity",
                                      tiles = "OSM"
                                     )

# Display the map
city_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [16]:
# Narrow down cities that fit criteria and drop any results with null values

weather_filtered_df = city_data_df.loc[(city_data_df["Max Temp"] < 50)
                                       & (city_data_df["Max Temp"] > 20) 
                                       & (city_data_df["Cloudiness"] < 50)
                                       & (city_data_df["Wind Speed"] < 50)
                                       & (city_data_df["Humidity"] < 90)
                                      ]

# Drop any rows with null values
weather_filtered_df.dropna()

# Display sample data
weather_filtered_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
6,6,ushuaia,-54.8000,-68.3000,31.66,80,20,8.05,AR,1690155121
9,9,grytviken,-54.2811,-36.5092,20.88,77,29,8.66,GS,1690155108
10,10,longyearbyen,78.2186,15.6401,49.84,76,40,12.66,SJ,1690155173
19,19,punta arenas,-53.1500,-70.9167,30.34,83,8,6.51,CL,1690154912
49,49,benalla,-36.5500,145.9833,46.29,77,1,2.93,AU,1690155397
53,53,tiksi,71.6872,128.8694,47.39,63,4,4.23,RU,1690155174
93,93,iqaluit,63.7506,-68.5145,49.69,71,20,8.01,CA,1690154975
291,291,mossel bay,-34.1831,22.1460,48.13,84,24,11.83,ZA,1690155448
391,391,plettenberg bay,-34.0527,23.3716,47.97,89,49,5.14,ZA,1690155469
405,405,el calafate,-50.3408,-72.2768,20.95,75,5,1.79,AR,1690155168


### Step 3: Create a new DataFrame called `hotel_df`.

In [17]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = weather_filtered_df[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
6,ushuaia,AR,-54.8000,-68.3000,80,
9,grytviken,GS,-54.2811,-36.5092,77,
10,longyearbyen,SJ,78.2186,15.6401,76,
19,punta arenas,CL,-53.1500,-70.9167,83,
49,benalla,AU,-36.5500,145.9833,77,
53,tiksi,RU,71.6872,128.8694,63,
93,iqaluit,CA,63.7506,-68.5145,71,
291,mossel bay,ZA,-34.1831,22.1460,84,
391,plettenberg bay,ZA,-34.0527,23.3716,89,
405,el calafate,AR,-50.3408,-72.2768,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [20]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories": "accommodation.hotel",
          "apiKey":geoapify_key}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = row["Lng"]
    latitude = row["Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] =  f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params = params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
ushuaia - nearest hotel: Apart Hotel Aires del Beagle
grytviken - nearest hotel: No hotel found
longyearbyen - nearest hotel: The Vault
punta arenas - nearest hotel: Hotel Lacolet
benalla - nearest hotel: Hotel Haven
tiksi - nearest hotel: Арктика
iqaluit - nearest hotel: Frobisher Inn
mossel bay - nearest hotel: Protea Hotel Mossel Bay
plettenberg bay - nearest hotel: Milkwood Manor
el calafate - nearest hotel: Hostería Las Avutardas


,City,Country,Lat,Lng,Humidity,Hotel Name
6,ushuaia,AR,-54.8000,-68.3000,80,Apart Hotel Aires del Beagle
9,grytviken,GS,-54.2811,-36.5092,77,No hotel found
10,longyearbyen,SJ,78.2186,15.6401,76,The Vault
19,punta arenas,CL,-53.1500,-70.9167,83,Hotel Lacolet
49,benalla,AU,-36.5500,145.9833,77,Hotel Haven
53,tiksi,RU,71.6872,128.8694,63,Арктика
93,iqaluit,CA,63.7506,-68.5145,71,Frobisher Inn
291,mossel bay,ZA,-34.1831,22.1460,84,Protea Hotel Mossel Bay
391,plettenberg bay,ZA,-34.0527,23.3716,89,Milkwood Manor
405,el calafate,AR,-50.3408,-72.2768,75,Hostería Las Avutardas


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [21]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points("Lng", 
                                   "Lat", 
                                   geo = True,
                                   color = "City",
                                   alpha = 1.5,
                                   size = "Humidity",
                                   tiles = "OSM",
                                   hover_cols = ["Hotel Name", "Country"]
                                  )

# Display the map
hotel_map

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)